# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure all your path constants are **relative to** ***DATA_DIR*** and **NOT hard-coded** in your code.

In [ ]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

     |████████████████████████████████| 471kB 2.7MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp36-cp36m-linux_x86_64.whl size=263767 sha256=5ee3d95c26b62d88fd25aa92f8daaadc90da212d2579f4fa04d3ac4d0116a047
  Stored in directory: /root/.cache/pip/wheels/55/66/40/1779aa0a8eb66e088669befe286f695cdfe420ba91ce662127
Successfully built pytrec-eval
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=a3b4223ead093b054a1324817600569a73a23fe04c018d7350818cf11692c2cf
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/government.zip", "government.zip")

'government.zip'

In [ ]:
!unzip government.zip

In [ ]:
# imports
# Put all your imports here
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
from whoosh import qparser
import os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import wget
import nltk
from nltk.stem import *
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
DATA_DIR = "government"
#
# Put other path constants here
#
DOCUMENTS_DIR = os.path.join(DATA_DIR, "documents")
TOPIC_FILE = os.path.join(DATA_DIR, "gov.topics")
QRELS_FILE = os.path.join(DATA_DIR, "gov.qrels")
DESCR_FILE = os.path.join(DATA_DIR, "topics-with-full-descriptions.txt")

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]

I think Mean Average Precision (MAP) is always a good measure for search system performance. R-prec would also be good measure for search system performance for government web site 

### Q1 (b): Provide answer to Q1 (b) here [markdown cell]

The reason why I think that Rprec is good is because it is p@R where R is the number of relevant documents for the query, it also ranks the documents. 

## Question 2

### Q2 (a): Write your code below

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q2, your query parser in QP_Q2, and your searcher in SEARCHER_Q2
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()

    # create and return the index
    return index.create_in(indexDir, schema)


In [ ]:
mySchema_Q2 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer())) # most basic tokenizer, we need to update

# now, create the index at the path INDEX_DIR based on the new schema
myIndex_Q2 = createIndex(mySchema_Q2)

In [ ]:
def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

In [ ]:
filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]

In [ ]:
filesToIndex

In [ ]:
addFilesToIndex(myIndex_Q2, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
myQueryParser_Q2 = QueryParser("file_content", schema=myIndex_Q2.schema)
mySearcher_Q2 = myIndex_Q2.searcher()

In [ ]:
INDEX_Q2 = myIndex_Q2 # Replace None with your index for Q2
QP_Q2 = myQueryParser_Q2 # Replace None with your query parser for Q2
SEARCHER_Q2 = mySearcher_Q2 # Replace None with your searcher for Q2

In [ ]:
def pyTrecEval(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        # print(1)
        # print(query_measures)
        for measure, value in query_measures.items():
            if measure != "Rprec":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure != "Rprec":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))
        

### Q2 (b): Provide answer to Q2 (b) here [markdown cell]

In [ ]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser_Q2, mySearcher_Q2) 

Rprec                    1       0.0000
Rprec                    2       0.0000
Rprec                    4       0.0000
Rprec                    6       0.0000
Rprec                    7       0.0000
Rprec                    9       0.0000
Rprec                    10      0.0000
Rprec                    14      0.0000
Rprec                    16      0.0000
Rprec                    18      1.0000
Rprec                    22      0.0000
Rprec                    24      1.0000
Rprec                    26      0.3333
Rprec                    28      0.0000
Rprec                    all     0.1667


### Q2 (c): Provide answer to Q2(c) here [markdown cell]

The baseline Whoosh did terriblely on my chosen measure, only query 18, 24 did well; they are 1 for 18, 24. Result for other queries is almost 0 for every other query.

## Question 3

In [ ]:
def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1":
                print(i.rstrip())

In [ ]:
printRelName(TOPIC_FILE, QRELS_FILE, myQueryParser_Q2, mySearcher_Q2, "4")

---------------------------Topic_id and Topic_phrase----------------------------------
4 wireless communications
---------------------------Return documents----------------------------------
4 Q0 G00-99-2247765 0 16.449155 test
4 Q0 G00-85-1525415 1 13.364613 test
4 Q0 G00-05-1218739 2 12.956314 test
4 Q0 G00-09-0774298 3 11.781349 test
4 Q0 G00-56-4151981 4 11.367248 test
4 Q0 G00-21-2229498 5 10.743958 test
4 Q0 G00-98-4068688 6 10.464865 test
4 Q0 G00-47-2117970 7 10.213356 test
4 Q0 G00-67-0152545 8 8.392871 test
4 Q0 G00-06-1757034 9 6.431556 test
4 Q0 G00-78-2551063 10 3.955775 test
4 Q0 G00-84-0274223 11 2.068438 test
---------------------------Relevant documents----------------------------------
4 0 G00-03-2855342 1
4 0 G00-36-1275993 1
4 0 G00-47-2117970 1
4 0 G00-65-0162935 1


### Q3 (a): Provide answer to Q3 (a) here [markdown cell]

The query I choose is "wireless communications", whose topic_id is 4. The false positive I am going to analyze is "G00-85-1525415" which my search engine ranked the second, but it is irrelevant by human judgement. By analzying this document, "wireless" term appeared many times and "communication" appeared 1 or 2 times, the high frequency of wireless is the reason why it is high ranked. However, the content itself was mainly focused on the transportation, which is the reason why it was judged as an irrevelant document. it is false positive because the query term appeared many times in the context of focusing on system and testing instead of taking about wireless communication. This motivates me to improve my queryparser by value more when all the query terms appeared together once than as long as one query term appeared many times.

The false negative case I am going to analyze is "G00-36-1275993". By skimming through the document myself, I find term "Wireless" many times as well as the "Communication, Telecomunnications", which represent the same thing our query means "wireless communications", but they are not matched because of the tokenization. So I am going to improve our tokenizer to improve our results.


### Q3 (b): Write your code below

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [ ]:
myFilter1 = RegexTokenizer() | LowercaseFilter() | CustomFilter(WordNetLemmatizer().lemmatize)

In [ ]:
mySchema_Q3 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = myFilter1))

myIndex_Q3 = createIndex(mySchema_Q3)

In [ ]:
addFilesToIndex(myIndex_Q3, filesToIndex)

In [ ]:
myQueryParser_Q3 = QueryParser("file_content", schema=myIndex_Q3.schema, group=qparser.OrGroup)
mySearcher_Q3 = myIndex_Q3.searcher()

In [ ]:
pyTrecEvalMap(TOPIC_FILE, QRELS_FILE, myQueryParser_Q3, mySearcher_Q3) 

map                      1       0.0305
map                      2       0.3000
map                      4       0.5658
map                      6       0.0500
map                      7       0.1778
map                      9       0.0385
map                      10      0.3333
map                      14      1.0000
map                      16      0.1647
map                      18      1.0000
map                      19      0.5000
map                      22      0.0526
map                      24      1.0000
map                      26      0.1081
map                      28      0.0698
map                      all     0.3594


In [ ]:
INDEX_Q3 = myIndex_Q3 # Replace None with your index for Q3
QP_Q3 = myQueryParser_Q3 # Replace None with your query parser for Q3
SEARCHER_Q3 = mySearcher_Q3 # Replace None with your searcher for Q3

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

First of all, I changed the basic tokenizer to my own customerized fliter which include RegexTokenizer() | LowercaseFilter() | CustomFilter(WordNetLemmatizer().lemmatize). It improved the false negative cases where the matching words that starts with capital characters in the documents. 
Also, i changed queryparser by evaluting all the query terms appeared together more than one single query terms appeard many times, it helps with the false positive case where the one single query term appeared many times, but the document was talking something totally different from other query terms (in our case, wireless system in transporation is what the false positive document talking about, but "wireless communications" was what we look for)

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]

it definetly improved the result! there are more non-zero Rprec value for many queries, and also the overall performance is increased

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]

Luckily, in our case, the orignal good queries are still remaining their performance, and more other queries is getting better

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]

My ideas were good. first of all, the increase in performance result proves my ideas is improving the search system, and also the logic behind my improvement for false positive and false negative are reasonable.

## Question 4

In [ ]:
from whoosh.qparser import MultifieldParser
from whoosh.scoring import *
import whoosh


In [ ]:
def pyTrecEvalMap(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            # topic_phrase = topic_phrase.lower()
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            # C = searcher.collector(limit=10)
            topicResults = searcher.search(topicQuery, limit=None, )
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        # print(1)
        # print(query_measures)
        for measure, value in query_measures.items():
            if measure != "map":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure != "map":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

In [ ]:
myFilter2 = RegexTokenizer() | LowercaseFilter() | StopFilter() | StemFilter() | IntraWordFilter()

In [ ]:
mySchema_Q4 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = myFilter2))

myIndex_Q4 = createIndex(mySchema_Q4)

In [ ]:
addFilesToIndex(myIndex_Q4, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
myQueryParser_Q4 = QueryParser("file_content", schema=myIndex_Q4.schema, group=qparser.OrGroup)
myQueryParser_Q4.add_plugin(qparser.FuzzyTermPlugin())
mySearcher_Q4 = myIndex_Q4.searcher()


In [ ]:
pyTrecEvalMap(TOPIC_FILE, QRELS_FILE, myQueryParser_Q4, mySearcher_Q4) 

map                      1       0.0433
map                      2       0.5357
map                      4       0.5604
map                      6       0.0500
map                      7       0.1778
map                      9       0.0714
map                      10      0.2500
map                      14      1.0000
map                      16      0.1601
map                      18      1.0000
map                      19      0.5000
map                      22      0.0417
map                      24      1.0000
map                      26      0.1065
map                      28      0.0711
map                      all     0.3712


### Please answer the following questions here
(a) A clear list of all final modifications made.  
(b)  Why each modification was made – how did it help?  
(c)  The  final  MAP  performance  that  these  modifications  attained.

- FuzzyTermPlugin, it helped a bit
- Fliter combination from Q3, not really helped
- 

In [ ]:
INDEX_Q4 = myIndex_Q4 # Replace None with your index for Q4
QP_Q4 = myQueryParser_Q4 # Replace None with your query parser for Q4
SEARCHER_Q4 = mySearcher_Q4 # Replace None with your searcher for Q4

## Validation

In [ ]:
# Run the following cells to make sure your code returns the correct value types

In [ ]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [ ]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [ ]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

### Q4 Validation

In [ ]:
assert(isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert(isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
